<a href="https://colab.research.google.com/github/insoucyant/MachineLearning/blob/main/PricePredictiounUsingPyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d camnugent/california-housing-prices
!unzip california-housing-prices.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
Dataset URL: https://www.kaggle.com/datasets/camnugent/california-housing-prices
License(s): CC0-1.0
Archive:  california-housing-prices.zip
  inflating: housing.csv             


In [47]:
# https://www.kaggle.com/code/realshivarv/simple-housing-price-predictor-using-pytorch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import torch

In [48]:
housing_data = pd.read_csv('housing.csv')
housing_data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [49]:
print(housing_data.shape)
print(housing_data.info())
for i in housing_data.columns:
  print(f'Column name is: {i} and it has {len(housing_data[i].unique()) }unique values')

(20640, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB
None
Column name is: longitude and it has 844unique values
Column name is: latitude and it has 862unique values
Column name is: housing_median_age and it has 52unique values
Column name is: total_rooms and it has 5926unique values
Column n

Categorical Variables

In [11]:
pd.get_dummies(housing_data['ocean_proximity']).head()
# housing_data_OHE = pd.concat([housing_data, pd.get_dummies(housing_data['ocean_proximity'])], axis=1)
# housing_data_OHE.drop('ocean_proximity', axis=1, inplace=True)
# housing_data_OHE

,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
20046,False,True,False,False,False
3024,False,True,False,False,False
15663,False,False,False,True,False
20484,True,False,False,False,False
9814,False,False,False,False,True


In [50]:
cont_features =['housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income']
cat_features = ['ocean_proximity']

In [51]:
# Set Hyperparmeters
EPOCHS, BATCH_SIZE, LEARNING_RATE, L1_LAMBDA = 1000, 16, 0.01, 0.01
torch.random.manual_seed(0)

In [34]:

lbl_encoders = {}
for feature in cat_features:
  lbl_encoders[feature] = LabelEncoder()
  housing_data[feature] = lbl_encoders[feature].fit_transform(housing_data[feature])
housing_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,3
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,3
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,3
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,3
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,3
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,1
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,1
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,1
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,1


In [36]:
# Stacking and converting to Tensors
cat_array = np.stack([housing_data[col] for col in cat_features], 1)
cat_array

array([[3],
       [3],
       [3],
       ...,
       [1],
       [1],
       [1]])

In [37]:
#convert numpy to tensors
cat_tensor = torch.tensor(cat_array, dtype=torch.int64)
cat_tensor

tensor([[3],
        [3],
        [3],
        ...,
        [1],
        [1],
        [1]])

In [42]:
# Stacking continuous features to tensors
cont_array = np.stack([housing_data[col] for col in cont_features], 1)
cont_tensors = torch.tensor(cont_array, dtype=torch.float)
cont_tensors.dtype

torch.float32

In [43]:
y = torch.tensor(housing_data['median_house_value'].values, dtype=torch.float).reshape(-1,1)
y

tensor([[452600.],
        [358500.],
        [352100.],
        ...,
        [ 92300.],
        [ 84700.],
        [ 89400.]])

In [44]:
cat_tensor.shape, cont_tensors.shape, y.shape

(torch.Size([20640, 1]), torch.Size([20640, 8]), torch.Size([20640, 1]))